In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
xml1 = """
<rule id="BRIEFCATCH_72217380358443072298334619098248039878" name="BRIEFCATCH_PUNCHINESS_921">
    <pattern>
        <token inflected="yes">call</token>
        <token>upon</token>
    </pattern>
    <antipattern>
        <token>calling</token>
        <token>upon</token>
    </pattern>
    <message>Would shorter words add punch?|**Example** from Justice Gorsuch: “When **called on** to interpret a statute, this Court generally seeks to discern and apply the ordinary meaning of its terms at the time of their adoption.”|**Example** from Deanne Maynard: “The [order] merely confirms that it was not until later proceedings that he was **called on** to single out these waters.”</message>
    <suggestion><match no="1"/> on</suggestion>
    <suggestion><match no="1" postag="(V.*)" postag_regexp="yes" postag_replace="$1">ask</match></suggestion>
    <short>{"ruleGroup":null,"ruleGroupIdx":0,"isConsistency":false,"isStyle":true,"correctionCount":2,"priority":"2.84","WORD":true,"OUTLOOK":true}</short>
    <example correction="called on|asked">She was <marker>called upon</marker> three times.</example>
</rule>
"""
xml2 = """
<rule id="BRIEFCATCH_72217380358443072298334619098248039878" name="BRIEFCATCH_PUNCHINESS_921">
    <pattern>
        <token inflected="yes">call</token>
        <token inflected="yes">FAKE</token>
        <token inflected="yes">TOKENs</token>
    </pattern>
    <pattern>
        <token inflected="yes">123</token>
        <token inflected="yes">TOKENs</token>
    </pattern>
    <message>Would shorter words add punch?|**Example** from Justice Gorsuch: “When **called on** to interpret a statute, this Court generally seeks to discern and apply the ordinary meaning of its terms at the time of their adoption.”|**Example** from Deanne Maynard: “The [order] merely confirms that it was not until later proceedings that he was **called on** to single out these waters.”</message>
    <suggestion><match no="1"/> on</suggestion>
    <suggestion><match no="1" postag="(V.*)" postag_regexp="yes" postag_replace="$1">ask</match></suggestion>
    <short>{"ruleGroup":null,"ruleGroupIdx":0,"isConsistency":false,"isStyle":true,"correctionCount":2,"priority":"2.84","WORD":true,"OUTLOOK":true}</short>
    <example correction="called on|asked">She was <marker>called upon</marker> three times.</example>
</rule>
"""

In [3]:
# from utils.dynamic_prompting import (
#     get_pos_tag_dicts_from_rule,
#     remove_message_and_short_tags,
#     rule_has_regex,
#     replace_all_instances_of_tag,
# )

# print(replace_all_instances_of_tag("pattern", xml1, xml2))

In [4]:
import openai
import os
from dotenv import load_dotenv


load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
from utils.dynamic_rule_checking import *
import re

re.findall(r"<rule.*?</rule>", xml2, re.DOTALL)[0]

/Users/consult/miniconda3/envs/briefcatch/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/consult/miniconda3/envs/briefcatch/lib/python3.9/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


'<rule id="BRIEFCATCH_72217380358443072298334619098248039878" name="BRIEFCATCH_PUNCHINESS_921">\n    <pattern>\n        <token inflected="yes">call</token>\n        <token inflected="yes">FAKE</token>\n        <token inflected="yes">TOKENs</token>\n    </pattern>\n    <pattern>\n        <token inflected="yes">123</token>\n        <token inflected="yes">TOKENs</token>\n    </pattern>\n    <message>Would shorter words add punch?|**Example** from Justice Gorsuch: “When **called on** to interpret a statute, this Court generally seeks to discern and apply the ordinary meaning of its terms at the time of their adoption.”|**Example** from Deanne Maynard: “The [order] merely confirms that it was not until later proceedings that he was **called on** to single out these waters.”</message>\n    <suggestion><match no="1"/> on</suggestion>\n    <suggestion><match no="1" postag="(V.*)" postag_regexp="yes" postag_replace="$1">ask</match></suggestion>\n    <short>{"ruleGroup":null,"ruleGroupIdx":0,"is

In [9]:
testxml = """
<rule id="BRIEFCATCH_295444651289759397744766776595353416512" name="BRIEFCATCH_PUNCHINESS_692">
    <pattern>
        <token>
            <exception regexp="yes">about|abroad|absolutely|after|ago|alike|alone|also|anywhere|as|as|been|before|below|clearly|completely|could|down|easily|even|far|here|hereby|herein|hereto|heretofore|hereunder|if|late|left|less|less|likely|little|might|more|more|most|much|nationwide|not|not|nothing|now|obviously|offshore|once|only|onward|out|patently|percent|previously|prior|pro|rather|right|same|since|so|strange|there|thereby|thereof|thereto|today|too|totally|up|upward|way|well|will|would|yet</exception>
        </token>
        <token regexp="yes">already|also|always|ever|formally|long|never|now|often|recently|repeatedly|specifically</token>
        <token regexp="yes">had|has|have</token>
        <token postag="VBN"/>
    </pattern>
    <message>Splitting the verb phrase is usually more natural.|**Example** from Chief Justice Roberts: “[P]laintiffs **have largely chosen** not to defend the decision below on that ground.”|**Example** from Justice Scalia: “The core of this right **has historically been**, and remains today . . . .”|**Example** from Linda Coberly: “This Court **has long recognized** that only Congress may create exemptions from federal statutes, and it **has never recognized** the judicial exemption that Petitioners now seek.”|**Example** from Maureen Mahoney and Greg Garre: “This Court **has never come close** to recognizing such a remarkable position.”</message>
    <suggestion><match no="1"/> <match no="3"/> <match no="2"/> <match no="4"/></suggestion>
    <short>{"ruleGroup":null,"ruleGroupIdx":0,"isConsistency":false,"isStyle":true,"correctionCount":1,"priority":"4.3932","WORD":true,"OUTLOOK":true}</short>
    <example correction="And had repeatedly shown"><marker>And repeatedly had shown</marker> that the prosecutor overstepped.</example>
    <example correction="I don't have input">I don't <marker>have input</marker> on this topic</example>
</rule>
"""

In [10]:
resp, xml = check_rule_modification(testxml)

In [11]:
print(resp)

<thoughts>The rule has 1 pattern and 1 corresponding example tag, which is correct. However, the example tag associated with the pattern does not follow the structure defined in the instructions. The marker tags in the example tag do not span the section of text represented by the pattern, and the correction attribute does not reflect the suggestion tag properly. The suggestion tag is represented in the correction field of the example tag, but the example itself is incorrect. The rule needs to be corrected to match the pattern with the example and to ensure the example tag's correction attribute reflects the suggestion tag properly.</thoughts>
<rule id="BRIEFCATCH_295444651289759397744766776595353416512" name="BRIEFCATCH_PUNCHINESS_692">
    <pattern>
        <token>
            <exception regexp="yes">about|abroad|absolutely|after|ago|alike|alone|also|anywhere|as|as|been|before|below|clearly|completely|could|down|easily|even|far|here|hereby|herein|hereto|heretofore|hereunder|if|late|l

In [12]:
print(xml)

<rule id="BRIEFCATCH_295444651289759397744766776595353416512" name="BRIEFCATCH_PUNCHINESS_692">
    <pattern>
        <token>
            <exception regexp="yes">about|abroad|absolutely|after|ago|alike|alone|also|anywhere|as|as|been|before|below|clearly|completely|could|down|easily|even|far|here|hereby|herein|hereto|heretofore|hereunder|if|late|left|less|less|likely|little|might|more|more|most|much|nationwide|not|not|nothing|now|obviously|offshore|once|only|onward|out|patently|percent|previously|prior|pro|rather|right|same|since|so|strange|there|thereby|thereof|thereto|today|too|totally|up|upward|way|well|will|would|yet</exception>
        </token>
        <token regexp="yes">already|also|always|ever|formally|long|never|now|often|recently|repeatedly|specifically</token>
        <token regexp="yes">had|has|have</token>
        <token postag="VBN"/>
    </pattern>
    <message>Splitting the verb phrase is usually more natural.|**Example** from Chief Justice Roberts: “[P]laintiffs **have 